In [59]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications

# dimensions of our images.
img_width, img_height = 224, 224

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'training'
validation_data_dir = 'validation'
nb_train_samples = 180
nb_validation_samples = 60
epochs = 50
batch_size = 15


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))
    print(train_data.shape)
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


In [60]:
    save_bottlebeck_features()
    train_top_model()

Found 180 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
(180, 7, 7, 512)
Epoch 1/50
12/12 [==============================] - 0s 18ms/step - loss: 4.6598 - accuracy: 0.5222 - val_loss: 2.4174 - val_accuracy: 0.5000
Epoch 2/50
12/12 [==============================] - 0s 9ms/step - loss: 1.3726 - accuracy: 0.6111 - val_loss: 2.5859 - val_accuracy: 0.5000
Epoch 3/50
12/12 [==============================] - 0s 9ms/step - loss: 0.9466 - accuracy: 0.6722 - val_loss: 0.7375 - val_accuracy: 0.5833
Epoch 4/50
12/12 [==============================] - 0s 10ms/step - loss: 0.6978 - accuracy: 0.7056 - val_loss: 0.6310 - val_accuracy: 0.7000
Epoch 5/50
12/12 [==============================] - 0s 10ms/step - loss: 0.6013 - accuracy: 0.7333 - val_loss: 0.6126 - val_accuracy: 0.5667
Epoch 6/50
12/12 [==============================] - 0s 10ms/step - loss: 0.6579 - accuracy: 0.6944 - val_loss: 0.6150 - val_accuracy: 0.7000
Epoch 7/50
12/12 [==============================] - 0s 9ms

In [29]:
import pandas as pd
import matplotlib.pyplot as plt

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
pd.DataFrame({'accuracy': accuracy,'val_accuracy': val_accuracy}).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) 
plt.show()

NameError: name 'history' is not defined

In [66]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense

top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224


nb_train_samples = 180
nb_validation_samples = 60
epochs = 30
batch_size = 15

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet',include_top= False,input_shape=(img_width, img_height,3))

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:11]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy', metrics.Precision(), metrics.Recall(),metrics.TruePositives(),metrics.TrueNegatives()])
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=5,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
model.save(
    filepath = 'finetunedvgg',
)

Found 180 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/30
12/12 [==============================] - 4s 322ms/step - loss: 0.9037 - accuracy: 0.7389 - precision_2: 0.7471 - recall_2: 0.7222 - true_positives_1: 65.0000 - true_negatives_1: 68.0000 - val_loss: 0.5764 - val_accuracy: 0.6833 - val_precision_2: 0.7895 - val_recall_2: 0.5000 - val_true_positives_1: 15.0000 - val_true_negatives_1: 26.0000
Epoch 2/30
12/12 [==============================] - 3s 250ms/step - loss: 0.7347 - accuracy: 0.6611 - precision_2: 0.6495 - recall_2: 0.7000 - true_positives_1: 63.0000 - true_negatives_1: 56.0000 - val_loss: 0.6556 - val_accuracy: 0.5667 - val_precision_2: 0.7500 - val_recall_2: 0.2000 - val_true_positives_1: 6.0000 - val_true_negatives_1: 28.0000
Epoch 3/30
12/12 [==============================] - 3s 254ms/step - loss: 0.6454 - accuracy: 0.6556 - precision_2: 0.6400 - recall_2: 0.7111 - true_positives_1: 64.0000 - true_negatives_1: 54.0000 - val_loss: 0.5797 

12/12 [==============================] - 4s 299ms/step - loss: 0.0523 - accuracy: 0.9722 - precision_2: 0.9670 - recall_2: 0.9778 - true_positives_1: 88.0000 - true_negatives_1: 87.0000 - val_loss: 0.4388 - val_accuracy: 0.9000 - val_precision_2: 0.9000 - val_recall_2: 0.9000 - val_true_positives_1: 27.0000 - val_true_negatives_1: 27.0000
Epoch 25/30
12/12 [==============================] - 4s 305ms/step - loss: 0.1168 - accuracy: 0.9722 - precision_2: 0.9670 - recall_2: 0.9778 - true_positives_1: 88.0000 - true_negatives_1: 87.0000 - val_loss: 0.4197 - val_accuracy: 0.8833 - val_precision_2: 0.8485 - val_recall_2: 0.9333 - val_true_positives_1: 28.0000 - val_true_negatives_1: 25.0000
Epoch 26/30
12/12 [==============================] - 3s 286ms/step - loss: 0.1212 - accuracy: 0.9556 - precision_2: 0.9457 - recall_2: 0.9667 - true_positives_1: 87.0000 - true_negatives_1: 85.0000 - val_loss: 0.4942 - val_accuracy: 0.8667 - val_precision_2: 0.8056 - val_recall_2: 0.9667 - val_true_positi

In [57]:
model.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)     

In [64]:
from tensorflow.keras import models
load_model = models.load_model('finetunedvgg')
image = 
load_model.predict()

In [67]:
# build the VGG16 network
base_model = applications.VGG16(weights='imagenet',include_top= False,input_shape=(img_width, img_height,3))

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy', metrics.Precision(), metrics.Recall(),metrics.TruePositives(),metrics.TrueNegatives()])
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=5,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
model.save(
    filepath = 'finetunedvgg2',
)

Found 180 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/30
12/12 [==============================] - 3s 291ms/step - loss: 0.5115 - accuracy: 0.8667 - precision_3: 0.8438 - recall_3: 0.9000 - true_positives_2: 81.0000 - true_negatives_2: 75.0000 - val_loss: 0.9198 - val_accuracy: 0.7333 - val_precision_3: 0.8889 - val_recall_3: 0.5333 - val_true_positives_2: 16.0000 - val_true_negatives_2: 28.0000
Epoch 2/30
12/12 [==============================] - 3s 241ms/step - loss: 0.7553 - accuracy: 0.7556 - precision_3: 0.7674 - recall_3: 0.7333 - true_positives_2: 66.0000 - true_negatives_2: 70.0000 - val_loss: 0.4459 - val_accuracy: 0.8500 - val_precision_3: 0.8889 - val_recall_3: 0.8000 - val_true_positives_2: 24.0000 - val_true_negatives_2: 27.0000
Epoch 3/30
12/12 [==============================] - 3s 245ms/step - loss: 0.3358 - accuracy: 0.8278 - precision_3: 0.8172 - recall_3: 0.8444 - true_positives_2: 76.0000 - true_negatives_2: 73.0000 - val_loss: 0.4211

12/12 [==============================] - 3s 259ms/step - loss: 0.0810 - accuracy: 0.9667 - precision_3: 0.9565 - recall_3: 0.9778 - true_positives_2: 88.0000 - true_negatives_2: 86.0000 - val_loss: 0.4054 - val_accuracy: 0.8833 - val_precision_3: 0.8485 - val_recall_3: 0.9333 - val_true_positives_2: 28.0000 - val_true_negatives_2: 25.0000
Epoch 25/30
12/12 [==============================] - 3s 262ms/step - loss: 0.0800 - accuracy: 0.9611 - precision_3: 0.9663 - recall_3: 0.9556 - true_positives_2: 86.0000 - true_negatives_2: 87.0000 - val_loss: 0.4011 - val_accuracy: 0.8667 - val_precision_3: 0.8235 - val_recall_3: 0.9333 - val_true_positives_2: 28.0000 - val_true_negatives_2: 24.0000
Epoch 26/30
12/12 [==============================] - 3s 268ms/step - loss: 0.0845 - accuracy: 0.9667 - precision_3: 0.9773 - recall_3: 0.9556 - true_positives_2: 86.0000 - true_negatives_2: 88.0000 - val_loss: 0.4897 - val_accuracy: 0.8500 - val_precision_3: 0.7838 - val_recall_3: 0.9667 - val_true_positi

In [68]:
# build the VGG16 network
base_model = applications.VGG16(weights='imagenet',include_top= False,input_shape=(img_width, img_height,3))

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy', metrics.Precision(), metrics.Recall(),metrics.TruePositives(),metrics.TrueNegatives()])
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=5,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
model.save(
    filepath = 'finetunedvgg3',
)

Found 180 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/30
12/12 [==============================] - 3s 290ms/step - loss: 0.4102 - accuracy: 0.8889 - precision_4: 0.9070 - recall_4: 0.8667 - true_positives_3: 78.0000 - true_negatives_3: 82.0000 - val_loss: 0.7900 - val_accuracy: 0.8167 - val_precision_4: 0.8065 - val_recall_4: 0.8333 - val_true_positives_3: 25.0000 - val_true_negatives_3: 24.0000
Epoch 2/30
12/12 [==============================] - 3s 239ms/step - loss: 0.3426 - accuracy: 0.9000 - precision_4: 0.9000 - recall_4: 0.9000 - true_positives_3: 81.0000 - true_negatives_3: 81.0000 - val_loss: 0.7900 - val_accuracy: 0.8167 - val_precision_4: 0.8065 - val_recall_4: 0.8333 - val_true_positives_3: 25.0000 - val_true_negatives_3: 24.0000
Epoch 3/30
12/12 [==============================] - 3s 239ms/step - loss: 0.2133 - accuracy: 0.9111 - precision_4: 0.8854 - recall_4: 0.9444 - true_positives_3: 85.0000 - true_negatives_3: 79.0000 - val_loss: 0.7900

KeyboardInterrupt: 